[https://github.com/wiseaidev/rust-data-analysis](https://github.com/wiseaidev/rust-data-analysis/blob/main/1-iris-data-analysis-rust.ipynb)

[Polars: Blazingly fast DataFrames in Rust, Python, Node.js, R, and SQL](https://github.com/pola-rs/polars/blob/main/README.md)
<br/><br/>
[Python Polars: A Lightning-Fast DataFrame Library](https://realpython.com/polars-python/)
<br/><br/>
[Analyzing data using Polars in Python: an introduction](https://www.tensorscience.com/posts/analyzing-data-using-polars-in-python-an-introduction.html)

[An Introduction to Polars: Python's Tool for Large-Scale Data Analysis](https://www.datacamp.com/blog/an-introduction-to-polars-python-s-tool-for-large-scale-data-analysis)

In [2]:
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

In [3]:
:dep polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}

In [4]:
:dep ndarray = {version = "0.15.6"}

In [5]:
:dep smartcore = {version = "0.3.2"}

In [6]:
:show_deps // show Installed Dependencies

ndarray = {version = "0.15.6"}
plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
smartcore = {version = "0.3.2"}


In [7]:
use polars::prelude::*;
use polars::frame::DataFrame;
use std::path::Path;
use ndarray::{ArrayBase, DataMut, Dimension, concatenate, Axis};
use plotters::prelude::*;
use smartcore::linalg::basic::matrix::DenseMatrix;
use ndarray::prelude::*;

In [8]:
:doc DataFrame

polars_core::frame

pub struct DataFrame {
    pub(crate) columns: Vec<Series, Global>,
}


A contiguous growable collection of Series that have the same length.

Use declarationsAll the common tools can be found in crate::prelude (or in polars::prelude).

use polars_core::prelude::*; // if the crate polars-core is used directly
// use polars::prelude::*;      if the crate polars is used

InitializationDefaultA DataFrame can be initialized empty:

# use polars_core::prelude::*;
let df = DataFrame::default();
assert!(df.is_empty());

Wrapping a Vec<Series>A DataFrame is built upon a Vec<Series> where the Series have the same length.

# use polars_core::prelude::*;
let s1 = Series::new("Fruit", &["Apple", "Apple", "Pear"]);
let s2 = Series::new("Color", &["Red", "Yellow", "Green"]);

let df: PolarsResult<DataFrame> = DataFrame::new(vec![s1, s2]);

Using a macroThe df macro is a convenient method:

# use polars_core::prelude::*;
let df: PolarsResult<DataFrame> = df!("Fruit" => &["Apple", "Apple", "Pear"],
                                      "Color" => &["Red", "Yellow", "Green"]);

Using a CSV fileSee the polars_io::csv::CsvReader.

IndexingBy a numberThe Index<usize> is implemented for the DataFrame.

# use polars_core::prelude::*;
let df = df!("Fruit" => &["Apple", "Apple", "Pear"],
             "Color" => &["Red", "Yellow", "Green"])?;

assert_eq!(df[0], Series::new("Fruit", &["Apple", "Apple", "Pear"]));
assert_eq!(df[1], Series::new("Color", &["Red", "Yellow", "Green"]));
# Ok::<(), PolarsError>(())

By a Series name# use polars_core::prelude::*;
let df = df!("Fruit" => &["Apple", "Apple", "Pear"],
             "Color" => &["Red", "Yellow", "Green"])?;

assert_eq!(df["Fruit"], Series::new("Fruit", &["Apple", "Apple", "Pear"]));
assert_eq!(df["Color"], Series::new("Color", &["Red", "Yellow", "Green"]));
# Ok::<(), PolarsError>(())

In [10]:
fn read_data_frame_from_csv(
    csv_file_path: &Path,
) -> DataFrame {
    CsvReader::from_path(csv_file_path)
        .expect("Cannot open file.")
        .has_header(true)
        .finish()
        .unwrap()
}

In [14]:
let iris_file_path: &Path = Path::new("../../../assets/data/Iris.csv");

In [15]:
let iris_df: DataFrame = read_data_frame_from_csv(iris_file_path);
iris_df

shape: (150, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa    │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa    │
│ …   ┆ …             ┆ …            ┆ …             ┆ …            ┆ …              │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3.0

In [16]:
:type iris_df

iris_df: DataFrame